问题 3 因蔬菜类商品的销售空间有限，商超希望进一步制定单品的补货计划，要求可 售单品总数控制在 27-33 个，且各单品订购量满足最小陈列量 2.5 千克的要求。根据 2023 年 6 月 24-30 日的可售品种，给出 7 月 1 日的单品补货量和定价策略，在尽量满足市场对各 品类蔬菜商品需求的前提下，使得商超收益最大。

* 和问题2的解题思路一样，只是从品类细化到单品，然后优化的时候多了总数 27-33 和最小陈列量的约束

In [4]:
import pandas as pd  # 导入pandas库用于数据处理
import numpy as np  # 导入numpy库用于数值计算
import seaborn as sns  # 导入seaborn库用于数据可视化
import matplotlib.pyplot as plt  # 导入matplotlib库用于绘图
import scipy.stats as stats  # 导入scipy库用于科学计算和统计
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为中文宋体以支持中文显示
plt.rcParams['axes.unicode_minus'] = False  # 设置正常显示负号
import warnings  # 导入warnings库用于警告控制
warnings.filterwarnings("ignore")  # 忽略警告信息

from sklearn.ensemble import RandomForestRegressor  # 导入随机森林回归模型
from sklearn.model_selection import cross_val_score  # 导入交叉验证工具
from statsmodels.tsa.stattools import adfuller  # 导入ADF检验用于时间序列稳定性检验
from statsmodels.tsa.arima.model import ARIMA  # 导入ARIMA模型
from sklearn.preprocessing import MinMaxScaler  # 导入MinMaxScaler用于数据标准化
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # 导入自相关和偏自相关图绘制工具
from statsmodels.stats.diagnostic import acorr_ljungbox  # 导入Ljung-Box检验用于时间序列白噪声检验
from datetime import datetime  # 导入datetime库用于日期和时间处理

ModuleNotFoundError: No module named 'statsmodels.tsa.arima'

In [8]:
df1 = pd.read_excel('附件1.xlsx')
df2 = pd.read_excel('附件2.xlsx')
df3 = pd.read_excel('附件3.xlsx')

In [9]:
#将附件123联合起来，得到每个单品的批发价格
# 合并两个DataFrame df2 和 df1
df = pd.merge(df2, df1)

# 创建一个空列表 ls 用于存储批发价格信息
ls = []

# 遍历 df 的单品编码和销售日期
for code, date in zip(df['单品编码'], df['销售日期']):
    # 查询 df3 中符合条件的行，即日期和单品编码匹配
    # 并获取对应行的批发价格信息，将其添加到列表 ls 中
    ls.append(df3[(df3.日期 == date) & (df3.单品编码 == code)]['批发价格(元/千克)'].values[0])

# 将列表 ls 中的批发价格信息添加为 df 的新列 '批发价格'
df['批发价格'] = ls

In [10]:
# 从DataFrame df 中排除退货商品和打折商品，仅保留销售类型为 '销售' 且是否打折销售为 '否' 的行
df = df[(df.是否打折销售 == '否') & (df.销售类型 == '销售')]

# 从筛选后的数据中选择销售日期大于等于指定日期的单品编码
# 并获取唯一的单品编码列表，表示这些单品在指定时间内销售
sales_product = df[df.销售日期 >= datetime.strptime('2023-06-24', '%Y-%m-%d')]['单品编码'].unique()

# 创建一个空的DataFrame df_pro 用于存储筛选后的数据
df_pro = pd.DataFrame()

# 遍历每个单品编码，筛选出相应的数据，并将其追加到 df_pro 中
for i in sales_product:
    tem = df[df.单品编码 == i]
    df_pro = pd.concat([df_pro, tem])

# 使用 groupby 对单品编码和销售日期进行分组，然后对销量、批发价格和销售单价进行聚合计算
sales = df_pro.groupby(['单品编码', '销售日期']).agg({"销量(千克)": [np.sum], '批发价格': ['mean'], '销售单价(元/千克)': [np.mean]})

# 重命名聚合后的列名
sales.columns = ['总销量', '批发价格', '销售价格']

# 计算总利润，即（销售价格 - 批发价格）* 总销量
sales['总利润'] = (sales['销售价格'] - sales['批发价格']) * sales['总销量']

# 计算成本利润率，即（销售价格 - 批发价格）/ 批发价格
sales['成本利润率'] = (sales['销售价格'] - sales['批发价格']) / sales['批发价格']

# 创建一个空列表 ls 用于存储单品的分类名称信息
ls = []

# 遍历 sales 的索引，即单品编码和销售日期的组合
for i in sales.index:
    # 从 df1 中根据单品编码获取对应的分类名称，并添加到列表 ls 中
    ls.append(df1[df1.单品编码 == i[0]]['分类名称'].values[0])

# 将列表 ls 中的分类名称信息添加为 sales 的新列 '分类名称'
sales['分类名称'] = ls

# 重置 sales 的索引，将原来的索引作为新的列 '单品编码' 和 '销售日期'
index = sales.index
sales = sales.reset_index()
sales['单品编码'] = [i[0] for i in index]
sales['销售日期'] = [i[1] for i in index]

# 返回包含构建好的单品成本、价格、销量和成本利润率信息的DataFrame sales
sales


NameError: name 'datetime' is not defined

In [11]:
# 创建空列表 ls 用于存储销售数据的归一化系数
# 创建空列表 values 用于存储销售数据的原始值
ls = []
values = []

# 遍历每个单品编码的唯一值，以计算每个单品的归一化系数
for i in sales['单品编码'].unique():
    tem = sales[sales.单品编码 == i]
    
    # 遍历每个销售日期的唯一值
    for o in tem['销售日期'].unique():
        t = tem[tem.销售日期 == o]
        s = t['总销量'].sum()
        
        # 计算每个销售数据行的归一化系数，然后添加到列表 ls 中
        for z in t['总销量'].values:
            ls.append(z / s)
        
        # 将每个销售数据行的原始值添加到列表 values 中
        for v in t.values:
            values.append(v)

# 使用列表 values 创建新的 DataFrame，列名保持不变
sales = pd.DataFrame(values, columns=sales.columns)

# 添加新的列 '归一化系数'，包含之前计算得到的归一化系数
sales['归一化系数'] = ls

# 计算加权批发价格、加权销售价格和加权成本利润率，分别乘以归一化系数
sales['加权批发价格'] = sales['归一化系数'] * sales['批发价格']
sales['加权销售价格'] = sales['归一化系数'] * sales['销售价格']
sales['加权成本利润率'] = sales['归一化系数'] * sales['成本利润率']

# 使用 groupby 对单品编码和销售日期进行分组，然后对总销量、总利润、加权批发价格、加权销售价格和加权成本利润率进行聚合计算
sales_pro = sales.groupby(['单品编码', '销售日期']).agg({"总销量": [np.sum], '总利润': [np.sum], '加权批发价格': [np.sum],
                                                     '加权销售价格': [np.sum], '加权成本利润率': [np.sum]})

# 重命名聚合后的列名
sales_pro.columns = ['总销量', '总利润', '加权批发价格', '加权销售价格', '加权成本利润率']

# 重置 sales_pro 的索引，将原来的索引作为新的列 '单品编码' 和 '销售日期'
index = sales_pro.index
sales_pro = sales_pro.reset_index()
sales_pro['单品编码'] = [i[0] for i in index]
sales_pro['销售日期'] = [i[1] for i in index]

# 返回包含加权信息的销售数据的DataFrame sales_pro
sales_pro


,单品编码,销售日期,总销量,总利润,加权批发价格,加权销售价格,加权成本利润率
0,102900005115250,2020-09-18,0.195,1.8720,14.00,23.6,0.685714
1,102900005115250,2020-09-27,1.308,13.7340,17.10,27.6,0.614035
2,102900005115250,2020-09-28,0.163,1.7115,17.10,27.6,0.614035
3,102900005115250,2020-11-22,44.844,349.7832,12.00,19.8,0.650000
4,102900005115250,2020-11-23,17.550,136.8900,12.00,19.8,0.650000
...,...,...,...,...,...,...,...
22692,106971533450003,2023-06-26,2.000,2.0800,1.96,3.0,0.530612
22693,106971533450003,2023-06-27,11.000,11.5500,1.95,3.0,0.538462
22694,106971533450003,2023-06-28,6.000,6.2400,1.96,3.0,0.530612
22695,106971533450003,2023-06-29,10.000,10.5000,1.95,3.0,0.538462


In [13]:
# 导入需要的库
from datetime import datetime
from chinese_calendar import is_holiday, is_workday

# 创建空列表 ls1、ls2 和 ls3，用于存储不同的日期信息
ls1 = []  # 存储是否工作日的标志
ls2 = []  # 存储是否节假日的标志
ls3 = []  # 存储季节信息的标志

# 遍历销售数据中的销售日期，并进行日期信息的提取和标记
for i in sales_pro['销售日期'].astype('str').values:
    # 使用 is_workday 函数判断日期是否工作日，将结果转换为整数后添加到 ls1 中
    ls1.append(int(is_workday(datetime.strptime(i, '%Y-%m-%d'))))
    
    # 使用 is_holiday 函数判断日期是否节假日，将结果转换为整数后添加到 ls2 中
    ls2.append(int(is_holiday(datetime.strptime(i, '%Y-%m-%d'))))
    
    # 提取日期中的月份信息，根据不同的月份进行季节的标记
    month = int(i[5:7])
    if month in [3, 4, 5]:
        ls3.append(0)  # 春季
    elif month in [6, 7, 8]:
        ls3.append(1)  # 夏季
    elif month in [9, 10, 11]:
        ls3.append(2)  # 秋季
    elif month in [12, 1, 2]:
        ls3.append(3)  # 冬季

# 将生成的日期信息添加到销售数据的新列中
sales_pro['是否工作日'] = ls1  # 是否工作日标志列
sales_pro['是否节假日'] = ls2  # 是否节假日标志列
sales_pro['春夏秋冬'] = ls3  # 季节标志列

# 创建新的列 '单品编码_code'，并初始化为 0
sales_pro['单品编码_code'] = 0

# 使用循环对每个唯一的单品编码进行编号
n = 0
for i in sales_pro['单品编码'].unique():
    # 将相同单品编码的行的 '单品编码_code' 列赋值为相应的编号
    sales_pro.loc[sales_pro.单品编码 == i, '单品编码_code'] = n
    n += 1

# 最终生成带有日期信息和单品编码编号的销售数据 DataFrame


In [28]:
# 导入 train_test_split 函数，用于数据集拆分
from sklearn.model_selection import train_test_split

# 定义特征列和目标列
x = ['加权销售价格', '单品编码_code', '加权批发价格', '是否工作日', '是否节假日', '春夏秋冬']
y = ['总销量']

# 使用 train_test_split 函数将数据集拆分为训练集和验证集
xtrain, xvalid, ytrain, yvalid = train_test_split(sales_pro[x].values, sales_pro[y].values,
                                                  test_size=0.2)

# 创建随机森林回归模型对象
model = RandomForestRegressor(random_state=15)

# 在训练集上拟合模型
model.fit(xtrain, ytrain)

# 输出模型在验证集上的评分
print(model.score(xvalid, yvalid))

# 创建新的随机森林回归模型对象
model = RandomForestRegressor(random_state=15)

# 在全数据集上拟合模型
model.fit(sales_pro[x].values, sales_pro[y].values)


0.5070507755297036


RandomForestRegressor(random_state=15)

In [21]:
# 创建空列表，用于存储单品编码和对应的预测结果
name = []
predict = []

# 遍历每个唯一的单品编码
for i in sales_pro['单品编码'].unique():
    # 从数据中筛选出特定单品编码的数据
    tem = sales_pro[sales_pro.单品编码==i]
    
    # 将单品编码添加到 name 列表中
    name.append(i)
    
    # 1. 读取批发价格数据
    data = tem['加权批发价格']
    data_origin = data.copy()

    # 2. 使用ADF检验数据是否稳定，如果不稳定则进行一阶差分
    result = adfuller(data)

    # 3. 对处理后的数据进行归一化
    scaler = MinMaxScaler()
    scaled_data = pd.DataFrame(scaler.fit_transform(data.values.reshape(-1, 1)))

    # 4. 通过AIC确定ARIMA模型参数
    aic_values = {}
    for p in range(4):
        for q in range(4):
            try:
                model = ARIMA(scaled_data, order=(p, 1, q))
                result = model.fit()
                aic_values[(p, 1, q)] = result.aic
            except:
                continue

    # 找到AIC值最小的参数
    min_aic = min(aic_values, key=aic_values.get)

    # 5. 使用ARIMA模型进行预测
    train_size = len(scaled_data) - 3
    train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]
    model = ARIMA(train_data, order=min_aic)
    result = model.fit()
    
    # 预测未来7天的数据
    predictions = result.forecast(steps=7)
    
    # 将预测结果逆归一化，并保存到 predict 列表中
    predict.append(scaler.inverse_transform(predictions.values.reshape(-1, 1)).reshape(1, -1)[0])



min aci: (3, 1, 3)
min aci: (3, 1, 2)
min aci: (3, 1, 3)
min aci: (1, 1, 0)
min aci: (1, 1, 2)
min aci: (1, 1, 2)
min aci: (1, 1, 3)
min aci: (3, 1, 2)
min aci: (0, 1, 1)
min aci: (1, 1, 1)
min aci: (0, 1, 0)
min aci: (0, 1, 1)
min aci: (3, 1, 2)
min aci: (3, 1, 0)
min aci: (3, 1, 2)
min aci: (2, 1, 2)
min aci: (0, 1, 0)
min aci: (0, 1, 0)
min aci: (1, 1, 3)
min aci: (0, 1, 0)
min aci: (2, 1, 1)
min aci: (0, 1, 3)
min aci: (0, 1, 3)
min aci: (0, 1, 0)
min aci: (2, 1, 1)
min aci: (3, 1, 2)
min aci: (2, 1, 1)
min aci: (2, 1, 3)
min aci: (2, 1, 2)
min aci: (0, 1, 1)
min aci: (1, 1, 1)
min aci: (0, 1, 2)
min aci: (2, 1, 3)
min aci: (0, 1, 2)
min aci: (0, 1, 1)
min aci: (0, 1, 2)
min aci: (1, 1, 1)
min aci: (0, 1, 1)
min aci: (0, 1, 2)
min aci: (3, 1, 3)
min aci: (2, 1, 2)
min aci: (3, 1, 0)
min aci: (2, 1, 2)
min aci: (0, 1, 0)
min aci: (2, 1, 2)
min aci: (3, 1, 3)
min aci: (0, 1, 2)
min aci: (0, 1, 2)


In [23]:
# 创建空列表用于存储日期相关信息
name = []  # 存储单品编码
date = ['2023-07-01', '2023-07-02', '2023-07-03', '2023-07-04',
       '2023-07-05', '2023-07-06', '2023-07-07']  # 预定义日期列表
ls1 = []  # 存储是否工作日信息
ls2 = []  # 存储是否节假日信息
ls3 = []  # 存储季节信息

# 遍历日期列表，获取日期相关信息
for i in date:
    # 判断是否是工作日，返回 1 或 0
    ls1.append(int(is_workday(datetime.strptime(i, '%Y-%m-%d'))))
    
    # 判断是否是节假日，返回 1 或 0
    ls2.append(int(is_holiday(datetime.strptime(i, '%Y-%m-%d'))))
    
    # 获取日期的月份，根据月份判断季节
    month = int(i[5:7])
    if month in [3, 4, 5]:
        ls3.append(0)  # 春季
    elif month in [6, 7, 8]:
        ls3.append(1)  # 夏季
    elif month in [9, 10, 11]:
        ls3.append(2)  # 秋季
    elif month in [12, 1, 2]:
        ls3.append(3)  # 冬季

# 创建空的数据框 dataset 用于存储结果
dataset = pd.DataFrame()

# 遍历每个唯一的单品编码，为每个单品编码创建日期相关信息并合并到 dataset 中
for index, i in enumerate(sales_pro['单品编码'].unique()):
    # 根据单品编码创建对应数量的单品编码
    name = [i for o in range(7)]
    
    # 预定义日期列表
    date = ['2023-07-01', '2023-07-02', '2023-07-03', '2023-07-04',
           '2023-07-05', '2023-07-06', '2023-07-07']
    
    ls1 = []  # 存储是否工作日信息
    ls2 = []  # 存储是否节假日信息
    ls3 = []  # 存储季节信息
    
    # 遍历日期列表，获取日期相关信息
    for i in date:
        # 判断是否是工作日，返回 1 或 0
        ls1.append(int(is_workday(datetime.strptime(i, '%Y-%m-%d'))))
        
        # 判断是否是节假日，返回 1 或 0
        ls2.append(int(is_holiday(datetime.strptime(i, '%Y-%m-%d'))))
        
        # 获取日期的月份，根据月份判断季节
        month = int(i[5:7])
        if month in [3, 4, 5]:
            ls3.append(0)  # 春季
        elif month in [6, 7, 8]:
            ls3.append(1)  # 夏季
        elif month in [9, 10, 11]:
            ls3.append(2)  # 秋季
        elif month in [12, 1, 2]:
            ls3.append(3)  # 冬季
    
    # 创建临时数据框，存储单品编码、销售日期和相关信息
    temp_df = pd.DataFrame({'单品编码': name, '销售日期': date, '加权批发价格': predict[index], '是否工作日': ls1, '是否节假日': ls2, '春夏秋冬': ls3})
    
    # 将临时数据框合并到 dataset 中
    dataset = pd.concat([temp_df, dataset])

# 创建 '单品编码_code' 列并进行编码
dataset['单品编码_code'] = 0
n = 0 
for i in sales_pro['单品编码'].unique():
    # 为每个唯一的单品编码赋予一个唯一的编码值
    dataset.loc[dataset.单品编码==i, '单品编码_code'] = n
    n += 1

# 显示结果数据框 dataset
dataset


,单品编码,销售日期,加权批发价格,是否工作日,是否节假日,春夏秋冬,单品编码_code
0,106971533450003,2023-07-01,1.961142,0,1,1,47
1,106971533450003,2023-07-02,1.968069,0,1,1,47
2,106971533450003,2023-07-03,1.968069,1,0,1,47
3,106971533450003,2023-07-04,1.968069,1,0,1,47
4,106971533450003,2023-07-05,1.968069,1,0,1,47
...,...,...,...,...,...,...,...
2,102900005115250,2023-07-03,15.459881,1,0,1,0
3,102900005115250,2023-07-04,15.181563,1,0,1,0
4,102900005115250,2023-07-05,15.366366,1,0,1,0
5,102900005115250,2023-07-06,15.363114,1,0,1,0


In [44]:
df4 = pd.read_excel('附件4.xlsx',sheet_name=1)

In [29]:
# 初始化空列表，用于存储不同定价策略下的结果
need_ls = []     # 存储补货量
price_ls = []    # 存储定价策略
reward_ls = []   # 存储预计收益

# 遍历数据集中的每一行
for index, value in enumerate(dataset[x[1:]].values):
    best_price = 0   # 最佳定价策略
    best_reward = 0  # 最佳预计收益
    best_need = 0    # 最佳补货量
    
    # 将数据转换为列表形式
    value = value.tolist()
    
    # 在一定范围内搜索最佳定价策略
    for price in np.arange(value[1] * 1.5, value[1] * 2, 0.1):
        i = [price] + value
        need = model.predict(np.array(i).reshape(1, -1))[0]  # 使用模型预测补货量
        reward = need * (i[0] - i[2])  # 根据补货量和定价计算预计收益
        
        # 更新最佳定价策略、补货量和预计收益
        if reward > best_reward:
            best_reward = reward
            best_price = price
            best_need = need
    
    # 将最佳结果添加到对应的列表中
    need_ls.append(best_need)
    price_ls.append(best_price)
    reward_ls.append(best_reward)

# 将计算结果添加到数据集中
dataset['补货量'] = need_ls
dataset['定价策略'] = price_ls
dataset['预计收益'] = reward_ls

# 将结果保存为 Excel 文件
dataset.to_excel('问题3\\价格和补货策略.xlsx', index=False)

In [48]:
dataset[dataset.销售日期=='2023-07-01'].sort_values(by='预计收益',ascending=False)[:33].to_excel('问题3\\价格和补货策略.xlsx',index=False)